In [47]:
# Read .litmus file into string

TESTNAME = "SB"
SIZEOFINT = 32
UNUSED_REG = "EDX"

import numpy as np
f=open(TESTNAME + ".litmus", "r")
string = f.read()
print(string)

X86 SB
"Fre PodWR Fre PodWR"
{ x=0; y=0; }
 P0          | P1          ;
 MOV [x],$1  | MOV [y],$1  ;
 MOV EAX,[y] | MOV EAX,[x] ;
locations [x;y;]
exists (0:EAX=0 /\ 1:EAX=0)


In [48]:
# Extract the initialization information (line in curly brackets) into init

init_start = string.index('{')
init_end = string.index('}')
init = string[init_start:init_end+1]

In [49]:
# Extract the instructions into instrs. First index is thread, second index is line

code_start = string[init_end:].index(';')+init_end+2
number_of_threads = string[init_end+2:code_start].count('|') + 1
number_of_lines = string[code_start:].count('|')
instrs = [[None]*number_of_lines for _ in range(number_of_threads)] 

instr_start = code_start
for i in range(number_of_lines):
    for j in range(number_of_threads):
        bar = string[instr_start:].find('|')
        if(bar<0):
            bar = np.inf
        instr_end = min(bar,string[instr_start:].find(';'))+instr_start
        instrs[j][i] = string[instr_start:instr_end]
        instr_start = instr_end+1       

In [50]:
# Extract the final condition into cond

code_end = instr_start
cond_start = string[code_end:].index('(') + code_end
cond_end = string[code_end:].index(')') + code_end
cond = string[cond_start:cond_end+1]

In [51]:
# Classify operations. Supported: reads, writes, fences. ops has the same structure as instrs.

ops = [[None]*number_of_lines for _ in range(number_of_threads)] 
for i in range(number_of_lines):
    for j in range(number_of_threads):
        if(instrs[j][i].find("MFENCE")>= 0):
            ops[j][i] = "fence"
        elif(instrs[j][i].find('[') < instrs[j][i].find(',')):
            ops[j][i] = "write"
        else:
            ops[j][i] = "read"        

In [52]:
# Extract memory locations used for writes, registers used for reads and unique write values, 
# to determine the needed shift amount

memlocs = set()
reglocs = set()
writevals = set()
for i in range(number_of_lines):
    for j in range(number_of_threads):
        instr = instrs[j][i]
        if(ops[j][i] == "write"):
            memlocs.update(instr[instr.find('[')+1:instr.find('[')+2])
            writevals.update(instr[instr.find('$')+1:instr.find('$')+2])
        if(ops[j][i] == "read"):
            reglocs.update([str(j) + instr[instr.find(',')-3:instr.find(',')]])
            
no_vals = len(writevals)
sh_amt = 1 + no_vals//2

In [56]:
# Write file with unrolling

# Open and write initial stuff
output_file = open(TESTNAME + "_wshifts.s", "w")
output_file.write(string[:(string[init_end:].index(';')+init_end+1)])

# Initialize counters
output_file.write("\n MOV " + UNUSED_REG + ",$1  | MOV " + UNUSED_REG + ",$1  ;\n")

for i in range(SIZEOFINT):
    for j in range(number_of_lines):
        for k in range(number_of_threads):
            # If write, make sure we write counter instead of immediate
            if(ops[k][j] != "write"):
                output_file.write(instrs[k][j])
            else:
                output_file.write(instrs[k][j][:instrs[k][j].find('$')] + UNUSED_REG + " ")
                
            # Terminate appropriately
            if (k != number_of_threads - 1):
                output_file.write("|")
            else: 
                output_file.write(";")
   
    # Shift counter left
    if (i != SIZEOFINT-1):
        output_file.write("\n\n SLL " + UNUSED_REG + ",$" + str(sh_amt) + "  | SLL " + UNUSED_REG + ",$" + str(sh_amt) + "  ;\n")

# Write final stuff & close
output_file.write(string[code_end:])
output_file.close()